In [2]:
import config
import torch.optim as optim
from custom_models.YOLOv3.model import YOLOv3
from tqdm import tqdm

from custom_models.YOLOv3.utils import (
    mean_average_precision,
    get_evaluation_bboxes,
    check_class_accuracy,
    get_loaders,
    cells_to_bboxes,
    non_max_suppression,
    intersection_over_union
)


import torch
from custom_models.YOLOv3.loss import YoloLoss
loss_fn = YoloLoss()
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_loader, test_loader, train_eval_loader,test_dataset = get_loaders(
    train_csv_path=config.DATASET + "/train.csv", test_csv_path=config.DATASET + "/test.csv"
)

In [4]:
num_classes = 20
model = YOLOv3(num_classes=num_classes)
model.load_state_dict(torch.load("/home/deepanshu/Desktop/ERAV1/S13/lightning_version/Final_trained_model.pth"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/deepanshu/Desktop/ERAV1/S13/lightning_version/Final_trained_model.pth'

In [ ]:
model.to("cpu")
print("Transfrered")
#import config
#check_class_accuracy(model, test_loader, threshold=0.2)

Transfrered


In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
def plot_image(image, boxes):
    # Getting the color map from matplotlib
    class_labels = [
        "aeroplane",
        "bicycle",
        "bird",
        "boat",
        "bottle",
        "bus",
        "car",
        "cat",
        "chair",
        "cow",
        "diningtable",
        "dog",
        "horse",
        "motorbike",
        "person",
        "pottedplant",
        "sheep",
        "sofa",
        "train",
        "tvmonitor"
    ]
    colour_map = plt.get_cmap("tab20b")
    # Getting 20 different colors from the color map for 20 different classes
    colors = [colour_map(i) for i in np.linspace(0, 1, len(class_labels))]

    # Reading the image with OpenCV
    img = np.array(image)
    # Getting the height and width of the image
    h, w, _ = img.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)

    # Add image to plot
    ax.imshow(img)

    # Plotting the bounding boxes and labels over the image
    for box in boxes:
        # Get the class from the box
        class_pred = box[0]
        # Get the center x and y coordinates
        box = box[2:]
        # Get the upper left corner coordinates
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2

        # Create a Rectangle patch with the bounding box
        rect = patches.Rectangle(
            (upper_left_x * w, upper_left_y * h),
            box[2] * w,
            box[3] * h,
            linewidth=2,
            edgecolor=colors[int(class_pred)],
            facecolor="none",
        )
        
        # Add the patch to the Axes
        ax.add_patch(rect)
        
        # Add class name to the patch
        plt.text(
            upper_left_x * w,
            upper_left_y * h,
            s=class_labels[int(class_pred)],
            color="white",
            verticalalignment="top",
            bbox={"color": colors[int(class_pred)], "pad": 0},
        )

    # Display the plot
    plt.show()

In [9]:
import os
def inference(model, test_loader):
    count = 0
    k = 30
    #   anchors = config.ANCHORS
    anchors = config.ANCHORS
        
    misclf = list()
    classes = config.PASCAL_CLASSES
    os.makedirs("missclassified_images/", exist_ok=True)
    # while count<=10:    
    model.eval()
    x,labels = test_loader.dataset[1]
    x = x.to("cpu")
    predictions = model(x.unsqueeze(0))   
    bboxes = [[] for _ in range(x.shape[0])]

    batch_size = x.shape[0]
    bboxes = [[] for _ in range(batch_size)]
    for i in range(3):
        S = predictions[i].shape[2]
        anchor = torch.tensor([*anchors[i]]).to("cpu") * S
        boxes_scale_i = cells_to_bboxes(
            predictions[i], anchor, S=S, is_preds=True
        )
        for idx, (box) in enumerate(boxes_scale_i):
            bboxes[idx] += box
    
    nms_boxes = non_max_suppression(bboxes[idx], iou_threshold=0.4, threshold=0.4)
    plot_image(x.permute(1,2,0).detach().cpu(), nms_boxes)

In [10]:
import os

grid_size =(5,5)
mean = [0.485, 0.456, 0.406]
std = [1,1,1]
device = "cpu"
inference(model, test_loader)

KeyboardInterrupt: 

In [8]:
from torchvision import transforms
import cv2
import os
def Trueinference(model, input_img):
    anchors = config.ANCHORS
    transform_norm = transforms.Compose([transforms.ToTensor(), 
    transforms.Resize((416,416))])
    # get normalized image
    input_img = transform_norm(input_img).float()

    model.eval()
    predictions = model(input_img.unsqueeze(0))   
    #input_img = input_img.permute(1,2,0)

    bboxes = [[] for _ in range(input_img.shape[0])]

    batch_size = input_img.shape[0]
    bboxes = [[] for _ in range(batch_size)]
    for i in range(3):
        S = predictions[i].shape[2]
        anchor = torch.tensor([*anchors[i]]).to("cpu") * S
        boxes_scale_i = cells_to_bboxes(
            predictions[i], anchor, S=S, is_preds=True
        )
        for idx, (box) in enumerate(boxes_scale_i):
            bboxes[idx] += box
    
    nms_boxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.5)
    plot_image(input_img.permute(1,2,0).detach().cpu(), nms_boxes)

from PIL import Image
Trueinference(model,Image.open("/home/deepanshu/Desktop/ERAV1/S13/PASCAL_VOC/images/000001.jpg").convert('RGB'))


FileNotFoundError: [Errno 2] No such file or directory: '/home/deepanshu/Desktop/ERAV1/S13/PASCAL_VOC/images/000001.jpg'